## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,48.92,86,100,2.10,light rain
1,1,Byron Bay,AU,-28.6500,153.6167,60.35,79,84,13.02,broken clouds
2,2,Provideniya,RU,64.3833,-173.3000,46.27,77,96,6.80,overcast clouds
3,3,Adrar,MR,20.5022,-10.0711,94.96,28,93,10.96,overcast clouds
4,4,Kantemirovka,RU,49.7089,39.8592,59.45,94,19,7.07,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Adrar,MR,20.5022,-10.0711,94.96,28,93,10.96,overcast clouds
6,6,Baghdad,IQ,33.3406,44.4009,84.11,23,0,6.91,clear sky
12,12,Riyadh,SA,24.6877,46.7219,93.34,17,96,7.27,overcast clouds
26,26,Perry,US,37.2506,-83.1999,90.84,47,7,3.40,light rain
35,35,Bethel,US,41.3712,-73.4140,90.77,53,0,4.61,clear sky
36,36,Cockburn Town,TC,21.4612,-71.1419,82.36,65,40,18.41,scattered clouds
42,42,Albany,US,42.6001,-73.9662,91.17,47,19,3.40,few clouds
46,46,Itaituba,BR,-4.2761,-55.9836,86.16,66,20,0.00,few clouds
56,56,Kashan,IR,33.9831,51.4364,88.14,26,22,9.40,few clouds
58,58,Hithadhoo,MV,-0.6000,73.0833,81.25,72,100,4.36,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                156
City                   156
Country                156
Lat                    156
Lng                    156
Max Temp               156
Humidity               156
Cloudiness             156
Wind Speed             156
Current Description    156
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## No empty rows found.
### If empty rows were found, use code: clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Adrar,MR,94.96,overcast clouds,20.5022,-10.0711,
6,Baghdad,IQ,84.11,clear sky,33.3406,44.4009,
12,Riyadh,SA,93.34,overcast clouds,24.6877,46.7219,
26,Perry,US,90.84,light rain,37.2506,-83.1999,
35,Bethel,US,90.77,clear sky,41.3712,-73.4140,
36,Cockburn Town,TC,82.36,scattered clouds,21.4612,-71.1419,
42,Albany,US,91.17,few clouds,42.6001,-73.9662,
46,Itaituba,BR,86.16,few clouds,-4.2761,-55.9836,
56,Kashan,IR,88.14,few clouds,33.9831,51.4364,
58,Hithadhoo,MV,81.25,overcast clouds,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
# Hotel Name was blank, but not dropping. Switched to identifying blanks and dropped those rows.
# Otherwise, the following code would be used: hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ""].index)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Baghdad,IQ,84.11,clear sky,33.3406,44.4009,Baghdad Hotel
12,Riyadh,SA,93.34,overcast clouds,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
26,Perry,US,90.84,light rain,37.2506,-83.1999,Hampton Inn & Suites Hazard
35,Bethel,US,90.77,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
36,Cockburn Town,TC,82.36,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
...,...,...,...,...,...,...,...
679,Wewak,PG,82.02,scattered clouds,-3.5534,143.6268,In Wewak Boutique Hotel
690,Umm Lajj,SA,85.08,overcast clouds,25.0213,37.2685,Moon light Furnished Units
694,Baracoa,CU,80.37,overcast clouds,20.3467,-74.4958,Hostal Rio Miel
696,Cortez,US,90.10,broken clouds,37.3489,-108.5859,Sand Canyon National 9 Inn


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))